# Imports:

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import os
import librosa
import asyncio
import numpy as np
import pandas as pd
import scipy.stats as stats
import time
import nest_asyncio
from speechpy import feature

# Extract frequency features:

## Get metadata:

In [51]:
current_path = os.getcwd()
source = os.path.join(current_path, "CommonVoice")

metadata = pd.read_csv(source + "/train.csv", delimiter='	')
data = metadata[['path','gender']]
data_male = data.loc[data['gender'] == 'male']
data_male = data_male.sample(frac=1).reset_index(drop=True)
data_female = data.loc[data['gender'] == 'female']

data = pd.concat([data_male,data_female])
data= data.sample(frac=1).reset_index(drop=True)
data['path'] = data['path'].str.replace('.mp3','.wav')

data_dict = dict(zip(data.path, data.gender))
data_dict['common_voice_pt_22499040.wav']

<ipython-input-51-2f0ddbd1084b>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  data['path'] = data['path'].str.replace('.mp3','.wav')


'male'

## Extract features:

In [82]:
async def get_frequencies(count, file):
    keys = data_dict.keys()
    if file.endswith('.wav') and file in list(keys):
        file_path = os.path.join(current_path,"CommonVoice", "Full", file)

        if data_dict[file] == 'female':
            gender = 0
            print(gender)
        if data_dict[file] == 'male': 
            gender = 1
            print(gender)
  
        audio_data, sample_rate = await librosa.load(file_path)
        

        step = int(sample_rate/5) #3200 sampling points every 1/5 sec
        window_frequencies = []

        for i in range(0,len(audio_data),step):
            ft = np.fft.fft(audio_data[i:i+step]) #fft returns the list N complex numbers
            freqs = librosa.fft_frequencies(sr=16000, n_fft=len(ft))
            freqs = np.fft.fftfreq(len(ft)) #fftq tells you the frequencies associated with the coefficients
            imax = np.argmax(np.abs(ft))
            freq = freqs[imax]
            freq_in_hz = abs(freq *sample_rate)
            window_frequencies.append(freq_in_hz)
    print(window_frequencies)
    return window_frequencies, gender, file

In [83]:
async def get_features(count, file):
    async with sem:
        
        frequencies, gender, file_name = await get_frequencies(count, file)

        nobs, minmax, mean, variance, skew, kurtosis =  stats.describe(frequencies)
        median   = np.median(frequencies)
        mode     = stats.mode(frequencies).mode[0]
        std      = np.std(frequencies)
        low,peak = minmax
        q75,q25  = np.percentile(frequencies, [75 ,25])
        iqr      = q75 - q25

        features_list.append([file_name, nobs, mean, skew, kurtosis, median, mode, std, low, peak, q25, q75, iqr, gender])

        

        return nobs, mean, skew, kurtosis, median, mode, std, low, peak, q25, q75, iqr

In [84]:
# #Calculo de tempo de disparo
start_time = time.time()

#inicio do Loop
loop = asyncio.get_event_loop()

#Controle de requisições por vez
sem = asyncio.Semaphore(2000)

#Array de tasks
sents = []

nest_asyncio.apply()

#Coleta as recomendações para envio
gender_list = []
file_list = []
features_list = []


current_path = os.getcwd()
file_path = os.path.join(current_path,"CommonVoice", "Full")
audio_files = os.listdir(file_path)

for k, file in enumerate(audio_files):
    sent = asyncio.ensure_future(get_features(count=k+1, file=file))
    #sent = asyncio.create_task(get_features(count=k+1, files=audio_files))
    
    sents.append(sent)
    break
done, _ = loop.run_until_complete(asyncio.wait(sents))

In [85]:
sents

[<Task finished name='Task-55' coro=<get_features() done, defined at <ipython-input-83-0185601b6606>:1> exception=UnboundLocalError("local variable 'window_frequencies' referenced before assignment")>]

In [10]:
dataframe_features = pd.DataFrame(features_list, columns = ['FileName', 'nobs', 'mean', 'skew', 'kurtosis', 'median', 'mode', 'std', 'low', 'peak', 'q25', 'q75', 'iqr', 'Gender'])
dataframe_features

,FileName,nobs,mean,skew,kurtosis,median,mode,std,low,peak,q25,q75,iqr,Gender


In [ ]:
dataframe_features.to_csv('data/CommonVoice_Features_data.csv', index=False)